# Visualisation of Latent Representations with Autoencoder and UMAP

In this project, we train an autoencoder to learn latent representations of images. We use DBSCAN to cluster the latents and UMAP to visualise them in a 2-dimensional space. We also include the projection of new images to the generated space.

In [44]:
#Importing the libraries

from PIL import Image, ImageDraw
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models
import umap
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
import pandas as pd

In [45]:
# We load the data

destination_directory = "data10"


def load_images(directory):
    """
    Loads images from the specified directory and their labels.

    :param directory: Path to the dataset directory.
    :return: Arrays of images and labels.
    """
    images = []
    labels = []
    paths = []
    classes = os.listdir(directory)
    for cls in classes:
        cls_dir = os.path.join(directory, cls)
        if os.path.isdir(cls_dir):
            for file in os.listdir(cls_dir):
                if file.endswith(".png"):
                    path = os.path.join(cls_dir, file)
                    image = Image.open(path).convert("L")  # Convert to grayscale
                    image = image.resize((64, 64))  # Resize
                    image = np.array(image) / 255.0  # Normalize
                    images.append(image.flatten())
                    labels.append(int(cls.split("_")[1]))
                    paths.append(path)
    return np.array(images), np.array(labels), paths


X, y, paths = load_images("data10")


In [ ]:
# Construction and Training of the Autoencoder

input_dim = X.shape[1]
encoding_dim = 64  

input_img = layers.Input(shape=(input_dim,))
encoded = layers.Dense(256, activation='relu')(input_img)
encoded = layers.Dense(128, activation='relu')(encoded)
encoded = layers.Dense(encoding_dim, activation='relu')(encoded)

decoded = layers.Dense(128, activation='relu')(encoded)
decoded = layers.Dense(256, activation='relu')(decoded)
decoded = layers.Dense(input_dim, activation='sigmoid')(decoded)

autoencoder = models.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

autoencoder.fit(X, X,
                epochs=100,  
                batch_size=256,
                shuffle=True,
                validation_split=0.2)


In [ ]:
# Feature Extraction and Clustering with DBSCAN


encoder = models.Model(input_img, encoded)
X_encoded = encoder.predict(X)


reducer = umap.UMAP(n_components=2, random_state=42)
X_umap = reducer.fit_transform(X_encoded)

dbscan = DBSCAN(eps=0.5, min_samples=10)
clusters_dbscan = dbscan.fit_predict(X_encoded)

from collections import Counter
cluster_to_squares = {
    -1: "Noise",  
    0: "2 Squares",
    1: "10 Squares",
    2: "5 Squares",
    3: "4 Squares",
    4: "3 Squares",
    5: "8 Squares",
    6: "6 Squares",
    7: "1 Squares",
    8: "7 Squares",
    9: "9 Squares"
}
cluster_counts = Counter(clusters_dbscan)
cluster_labels = {cluster: f"{cluster_to_squares[cluster]} ({count})" for cluster, count in cluster_counts.items()}

plt.figure(figsize=(12, 10))
sns.scatterplot(x=X_umap[:,0], y=X_umap[:,1], hue=[cluster_labels[c] for c in clusters_dbscan], palette='tab10', legend='full')
plt.title("Image Clusters using DBSCAN and UMAP")
plt.xlabel("UMAP 1")
plt.ylabel("UMAP 2")
plt.legend(title='Number of Squares (Count)', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()


In [64]:
# Function to locate new image in the UMAP

def update_umap_with_image(new_image, encoder, dbscan, reducer, X_umap, clusters_dbscan, cluster_to_squares):
    """
    Processes a new image, classifies it, and updates the UMAP plot.

    :param new_image: Image to classify.
    :param encoder: Trained encoder model.
    :param dbscan: Trained DBSCAN model.
    :param reducer: Trained UMAP model.
    :param X_umap: Existing UMAP data.
    :param clusters_dbscan: Existing clusters.
    :param cluster_to_squares: Dictionary mapping clusters to number of squares.
    """
    # Preprocess the new image
    new_image = new_image.resize((64, 64)).convert("L")
    new_image = np.array(new_image) / 255.0
    new_image_flat = new_image.flatten().reshape(1, -1)
    
    # Encode the image
    encoded_img = encoder.predict(new_image_flat)
    
    # Predict the cluster
    new_cluster = dbscan.fit_predict(encoded_img)[0]
    num_squares = cluster_to_squares.get(new_cluster, "Unknown")
    
    # Reduce dimensionality with UMAP
    new_image_umap = reducer.transform(encoded_img)
    
    # Update the UMAP plot
    plt.figure(figsize=(12, 10))
    sns.scatterplot(x=X_umap[:,0], y=X_umap[:,1], hue=[cluster_labels[c] for c in clusters_dbscan], palette='tab10', legend='full')
    plt.scatter(new_image_umap[:,0], new_image_umap[:,1], color='red', label=f'New Image: {num_squares} Squares')
    plt.title("Image Clusters using DBSCAN and UMAP")
    plt.xlabel("UMAP 1")
    plt.ylabel("UMAP 2")
    plt.legend(title='Number of Squares (Count)', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.show()


In [ ]:
# Selection and placement of the new image
new_image = Image.open('image_13.png')
update_umap_with_image(new_image, encoder, dbscan, reducer, X_umap, clusters_dbscan, cluster_to_squares)

In [67]:
# Save the autoencoder and the encoder
autoencoder.save('autoencoder_model10.h5')
encoder.save('encoder_model10.h5')